In [22]:
import numpy as np
import pandas as pd

import refinitiv.dataplatform.eikon as ek
import option_ric_tools as ort

ek.set_app_key('DEFAULT_CODE_BOOK_APP_KEY')

Let's start by defining what data we will be fetching. We will need to define the following:
* The ISIN for the underlying security as a string
* The maturity date for any options as a string
* The strike prices for which to fetch options prices as a list of integers
* The kinds of options (call/put) to fetch prices for as a list of strings (allowed values are 'C' and 'P')

An example for IBM is provided below. If you are unsure what strikes to fetch you might want to check the price of the underlying 90 days prior to your chosen maturity. Any configurations for which no prices are available will be skipped.

In [23]:
isin = 'US5949181045'
maturity = '2022-03-18'
strikes = [280, 290, 300, 310, 320, 330, 340, 350, 360]
kinds = ['C', 'P']

The code below will then try to fetch pricing for what we defined. Using the Eikon API requires quite a lot of preprocessing, which is taken care of in the file 'option_ric_tools.py' The price series for each option will be added as a column.

In [24]:
price_dfs = []
for kind in kinds:
    assert kind in ['C', 'P']
    for strike in strikes:
        kind_str = 'call' if kind == 'C' else 'P'
        print(f'\nFetching prices for {kind_str} option with strike price {strike}')
        rics, prices_list = ort.get_optionRic(isin, maturity, strike, kind)
        try:
            price_df = prices_list[0]
            price_df = price_df.drop(['BID', 'ASK'], axis=1)
            price_df.columns = [f'{kind}{strike}']
            price_dfs.append(price_df)
        except IndexError:
            print('Skipping empty combination')

price_df = pd.concat(price_dfs, axis=1)


Fetching prices for call option with strike price 280
Option RIC for OPQ exchange is successfully constructed

Fetching prices for call option with strike price 290
Option RIC for OPQ exchange is successfully constructed

Fetching prices for call option with strike price 300
Option RIC for OPQ exchange is successfully constructed

Fetching prices for call option with strike price 310
Option RIC for OPQ exchange is successfully constructed

Fetching prices for call option with strike price 320
Option RIC for OPQ exchange is successfully constructed

Fetching prices for call option with strike price 330
Option RIC for OPQ exchange is successfully constructed

Fetching prices for call option with strike price 340
Option RIC for OPQ exchange is successfully constructed

Fetching prices for call option with strike price 350
Option RIC for OPQ exchange is successfully constructed

Fetching prices for call option with strike price 360
Option RIC for OPQ exchange is successfully constructed



In [25]:
price_df.head()

,C280,C290,C300,C310,C320,C330,C340,C350,C360,P280,P290,P300,P310,P320,P330,P340,P350,P360
2021-03-18,<NA>,8.45,6.8,5.5,4.7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,93.64,<NA>,<NA>,<NA>,<NA>
2021-03-19,<NA>,8.95,6.7,5.35,4.7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2021-03-22,11.28,9.05,7.15,<NA>,5.0,<NA>,<NA>,<NA>,2.19,<NA>,<NA>,<NA>,<NA>,92.29,<NA>,<NA>,<NA>,<NA>
2021-03-23,10.8,9.5,7.6,6.18,<NA>,<NA>,<NA>,2.74,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2021-03-24,10.1,<NA>,6.6,<NA>,4.11,<NA>,<NA>,2.33,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,105.5,<NA>,<NA>


Let's now fetch the price series for the underlying for the same time period. Eikon uses a special identifier called RIC. The code below find the corresponding RIC for the ISIN we defined. It also find the earliest and latest dates in our options price series.

In [26]:
ric = ek.get_symbology(symbol=isin, from_symbol_type='ISIN', to_symbol_type='RIC').iloc[0,0]
sdate = str(price_df.index.min().date())
edate = str(price_df.index.max().date())

We can now fetch the price series for the underlying and add it as a column to the dataframe.

In [27]:
stock = ek.get_timeseries(
    rics=[ric],
    fields=['CLOSE'],
    start_date=sdate,
    end_date=edate,
    interval='daily'
)

stock = stock.reset_index()
stock.columns.name = None
stock.index = stock['Date']
stock.index.name = None
stock = stock.drop('Date', axis=1)
stock.columns = ['Underlying']

price_df = stock.join(price_df)

In [28]:
price_df.head()

,Underlying,C280,C290,C300,C310,C320,C330,C340,C350,C360,P280,P290,P300,P310,P320,P330,P340,P350,P360
2021-03-18,230.72,<NA>,8.45,6.8,5.5,4.7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,93.64,<NA>,<NA>,<NA>,<NA>
2021-03-19,230.35,<NA>,8.95,6.7,5.35,4.7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2021-03-22,235.99,11.28,9.05,7.15,<NA>,5.0,<NA>,<NA>,<NA>,2.19,<NA>,<NA>,<NA>,<NA>,92.29,<NA>,<NA>,<NA>,<NA>
2021-03-23,237.58,10.8,9.5,7.6,6.18,<NA>,<NA>,<NA>,2.74,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2021-03-24,235.46,10.1,<NA>,6.6,<NA>,4.11,<NA>,<NA>,2.33,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,105.5,<NA>,<NA>


As a final step we save the constructed dataframe for further analysis. The default code saves the dataframe in feather format, but if you can issues you can uncomment the lines further down to export to csv.

In [30]:
# Convert the 'Date' column to datetime format
price_df['Date'] = pd.to_datetime(price_df['Date'], format='%Y-%m-%d')

# Set the Date column as the index
price_df.set_index('Date', inplace=True)

# Export to CSV (ensure the date column is well formatted)
price_df.to_csv('price_data.csv', encoding='utf-8')

print("Data exported to price_data.csv")



Data exported to price_data.csv
